In [39]:
# !pip install deepface
from deepface import DeepFace

import matplotlib.pyplot as plt
import pandas as pd 
import csv

params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w"}
plt.rcParams.update(params)

result_df = pd.read_csv('./result/result_debface.csv')

In [40]:
result_df

,Model,Gender,Age,Accuracy,Total Images,Undetected
0,ArcFace,Woman,41,40.0,25,0
1,ArcFace,Man,41,100.0,89,0


In [6]:
# Select a desired model
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
model_used = "ArcFace"

In [44]:
# Image selection
dataset_dir = "../../../../Monash/C2001-Y3/sem1/FIT3163/others/Assignment/Project-DebFace/DebFace/datasets/AFAD/AFAD-new"

# Age
age = 30

# Man, Woman
gender = "Woman"

In [41]:
def deepface_get_accuracy(path, age, gender, model):
    
    gender_index = 0
    if gender == "Man":
        gender_index = 111
    elif gender == "Woman":
        gender_index = 112

    # set path to images based on age and gender
    path = f"{path}/{str(age)}/{str(gender_index)}"
    images_array = os.listdir(path)
    total_images = len(images_array)

    res = 0
    undetected = 0
    result_analysis = []

    # Analyse each image with for loop
    for img in images_array:
        path_img = f"{path}/{img}"

        # Image analysis of age and gender 
        try:
            obj = DeepFace.analyze(img_path = path_img, actions = ['gender'], enforce_detection=False)
            result_analysis.append({'image': img, 'Gender': obj['gender']})
            if obj['gender'] == 'Man' and gender == "Man":
                res += 1
            elif obj['gender'] == 'Woman' and gender == "Woman":
                res += 1
                
        except ValueError:
            undetected += 1

    # calculate accuracy
    acc = round(res/(total_images-undetected), 2) * 100
    
    return result_analysis, {'Model': model_used, 'Gender': gender, 'Age':age, 'Accuracy': acc, 'Total Images': total_images, 'Undetected': undetected}

In [45]:
result_analysis, result_acc = deepface_get_accuracy(dataset_dir, age, gender, model_used)

1/1 [==============================] - 0s 242ms/step


In [47]:
result_analysis

[{'image': '10077-0.png', 'Gender': 'Man'},
 {'image': '10129-0.png', 'Gender': 'Man'},
 {'image': '10143-0.png', 'Gender': 'Woman'},
 {'image': '10156-0.png', 'Gender': 'Man'},
 {'image': '10280-0.png', 'Gender': 'Man'},
 {'image': '10288-0.png', 'Gender': 'Man'},
 {'image': '10368-0.png', 'Gender': 'Man'},
 {'image': '10385-0.png', 'Gender': 'Man'},
 {'image': '10408-0.png', 'Gender': 'Woman'},
 {'image': '10483-0.png', 'Gender': 'Man'},
 {'image': '10484-0.png', 'Gender': 'Woman'},
 {'image': '10577-0.png', 'Gender': 'Man'},
 {'image': '10587-1.png', 'Gender': 'Woman'},
 {'image': '10611-0.png', 'Gender': 'Man'},
 {'image': '10684-0.png', 'Gender': 'Man'},
 {'image': '10685-0.png', 'Gender': 'Man'},
 {'image': '10702-1.png', 'Gender': 'Man'},
 {'image': '10708-0.png', 'Gender': 'Man'},
 {'image': '10816-0.png', 'Gender': 'Woman'},
 {'image': '10834-0.png', 'Gender': 'Woman'},
 {'image': '10840-0.png', 'Gender': 'Man'},
 {'image': '10841-0.png', 'Gender': 'Woman'},
 {'image': '11119-

In [46]:
result_df_new = pd.DataFrame([result_acc])

frames = [result_df, result_df_new]
result_df = pd.concat(frames)

result_df = result_df.reset_index()
result_df.drop(columns=['index'], inplace=True)
result_df

,Model,Gender,Age,Accuracy,Total Images,Undetected
0,ArcFace,Woman,41,40.0,25,0
1,ArcFace,Man,41,100.0,89,0
2,ArcFace,Man,30,99.0,250,0
3,ArcFace,Woman,30,29.0,250,0


In [32]:
# Write results to csv file
result_df.to_csv("./result/result_debface.csv", index=False)